<a href="https://colab.research.google.com/github/zach401/optimal_sizing_solar_ev_charging/blob/master/optimal_solar_sizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/zach401/acnportal.git
!pip install acnportal/.

In [0]:
!pip install cvxpy

In [0]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from acnportal import acndata
from datetime import datetime, timedelta
import pytz
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import math
import cvxpy as cp
from collections import defaultdict
import time

tz = pytz.timezone('America/Los_Angeles')

In [0]:
client = acndata.DataClient('DEMO_TOKEN')

In [0]:
def get_data(start, end, site):
  data = pd.DataFrame(client.get_sessions_by_time('jpl', start, end))
  data.sort_values(by='connectionTime', inplace=True)
  return data


def get_data_matrix(data):
  connection_time = [v.hour + v.minute/60 for v in data['connectionTime']]
  durations = [v.total_seconds()/3600 for v in data['disconnectTime'] - data['connectionTime']]
  energy = [v for v in data['kWhDelivered']]
  return np.array([connection_time, durations, energy]).T


def train_gmm(data):
  d = get_data_matrix(data)
  gmm = GaussianMixture(35, n_init=1)
  gmm.fit(d)
  return gmm


def sample_sessions(n_days, n_per_day, model):
  """ Each row is a session in the form arrival, duration, energy."""
  preds = []
  for d in range(n_days):
      pred = model.sample(n_per_day)[0]
      pred[:, 0] += 24*d
      preds.append(pred)
  preds = np.concatenate(preds)
  return np.stack(preds)


def process_sessions(sessions, period, max_charging_power):
  sessions[:, :2] = np.floor(sessions[:, :2] * 60 / period)
  sessions[:, 2] = np.minimum(sessions[:, 2], 
                              np.floor(sessions[:, 1]) * period / 60 * max_charging_power)
  mask = np.logical_and(sessions[:, 1].astype(int) > 0, 
                        sessions[:, 2] >= 0)
  return sessions[mask]


def optimize_solar(scenarios, solar_curve, solar_lcoe, tou_prices, demand_charge, 
                   period_len, max_charging_power):
  alpha = cp.Variable(nonneg=True)
  problems = [] 
  for j, scenario in enumerate(scenarios):
    print(f'Adding scenario {j}...')
    horizon = int(max(scenario[:, 0] + scenario[:, 1]))
    print(f'Horizon {horizon}...')
    agg_rates = defaultdict(list)
    constraints = []
    for i, session in enumerate(scenario):
      a, d = int(session[0]), int(session[1])
      rates = cp.Variable(d)
      constraints.append(rates <= max_charging_power)
      # print('Adding energy constraints...')
      constraints.append(cp.sum(rates) * period_len == session[2])
      for t in range(d):
        agg_rates[a + t].append(rates[t])
    
    peak = cp.Variable(nonneg=True)
    
    scenario_solar = solar_curve[:horizon]
    obj = solar_lcoe * alpha * sum(scenario_solar) * period_len
    grid_power = {}
    for t in agg_rates:
      grid_power[t] = cp.maximum(0, sum(agg_rates[t]) - alpha * scenario_solar[t])
      obj += tou_prices[t] * grid_power[t] * period_len
      constraints.append(peak >= grid_power[t])
    obj += demand_charge * peak
    p = cp.Problem(cp.Minimize(obj), constraints)
    problems.append(p)
  total_prob = sum(problems) 
  total_prob.solve(verbose=True)
  return {'average_cost': total_prob.value / len(scenarios),
          'alpha': alpha.value}


class SamSolar:
    def __init__(self, period, csv_source, index_name, col_name, 
                 scale=None, capacity=None):
        init_time = datetime(2018, 1, 1).astimezone()
        self.period = period
        self.csv = csv_source
        raw_solar = pd.read_csv(self.csv, index_col=index_name)
        raw_solar.index = init_time + pd.to_timedelta(raw_solar.index, 'h')
        raw_solar = raw_solar.resample('{0}T'.format(self.period)).pad()  # should switch to interpolate...
        self.scale=scale
        raw_solar = raw_solar[col_name].fillna(0).clip(lower=0)
        if self.scale is not None:
            if capacity is None:
                raw_solar = (raw_solar / raw_solar.max())*self.scale
            else:
                raw_solar = (raw_solar / capacity) * self.scale
        self.generation = raw_solar

    def get_generation(self, start, end):
        if end < datetime(2019, 1, 1).astimezone():
            return self.generation[start:end].values
        else:
            return np.concatenate([self.generation[start:].values, self.generation[:end.replace(year=2018)]])

In [0]:
training_start = tz.localize(datetime(2019, 1, 1))
training_end = tz.localize(datetime(2019, 2, 1))
data = get_data(training_start, training_end, 'jpl')
gmm = train_gmm(data)

In [0]:
period = 15
period_len = period / 60
max_len = 30 * 24 * int(60 / period)
max_charging_power = 6.656
scenarios = []
for _ in range(5):
  sessions = sample_sessions(7, 30, gmm)
  sessions = process_sessions(sessions, period, max_charging_power)
  scenarios.append(sessions)

In [0]:
solar_data = 'tmy_yearly_system_production.csv'
solar = SamSolar(period, solar_data, 'Hours since 00:00 Jan 1', 'System power generated (kW)')

In [127]:
from acnportal.signals.tariffs import TimeOfUseTariff
simulation_start_time = tz.localize(datetime(2019, 2, 1))


# TODO: Add actual solar curve file. 
solar_curve = solar.get_generation(simulation_start_time, 
                                   simulation_start_time + timedelta(days=30))
solar_lcoe = 0.04

tariff = TimeOfUseTariff('sce_tou_ev_4_march_2019')
demand_charge = tariff.get_demand_charge(simulation_start_time)
tou_prices = tariff.get_tariffs(simulation_start_time, max_len, period)

init_time = time.time()
res = optimize_solar(scenarios, solar_curve, solar_lcoe, 
                      tou_prices, demand_charge, period_len, 
                      max_charging_power)
print(time.time() - init_time)

Adding scenario 0...
Horizon 655...
Adding scenario 1...
Horizon 651...
Adding scenario 2...
Horizon 658...
Adding scenario 3...
Horizon 656...
Adding scenario 4...
Horizon 655...

ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +6.662e+02  -2.125e+05  +6e+05  4e-01  7e+00  1e+00  1e+01    ---    ---    1  1  - |  -  - 
 1  +1.351e+03  -4.319e+04  +2e+05  8e-02  2e+00  2e+00  5e+00  0.7615  2e-01   1  1  1 |  0  0
 2  +1.328e+03  -7.207e+03  +6e+04  1e-02  8e-01  6e-01  1e+00  0.7696  3e-02   1  1  1 |  0  0
 3  +1.244e+03  -4.270e+03  +4e+04  9e-03  5e-01  4e-01  9e-01  0.4317  2e-01   1  1  1 |  0  0
 4  +1.220e+03  -1.946e+03  +2e+04  5e-03  3e-01  2e-01  5e-01  0.6066  3e-01   1  1  1 |  0  0
 5  +1.161e+03  -3.370e+02  +1e+04  2e-03  1e-01  8e-02  3e-01  0.5620  9e-02   1  0  1 |  0  0
 6  +1.158e+03  -2.464e+02  +1e+04  2e-03  9e-02  7e-02  2

In [128]:
res

{'alpha': 119.74392587054452, 'average_cost': 223.46508927120448}

0.766779